<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Задача - выявить негативные комментарии на сайте интернет-магазина для последующей их модерации. Нужно создать модель,
 определяющую токсичность комментариев.

# Подготовка

In [ ]:
# <установка библиотек>
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install lightgbm
!pip install torch
!pip install transformers

In [1]:
# <импорт библиотек>
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import re

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords as nltk_stopwords

from sklearn.model_selection import GridSearchCV

# <дерево решений>
from sklearn.tree import DecisionTreeClassifier
# <случайный лес>
from sklearn.ensemble import RandomForestClassifier
# <логистическая регрессия>
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import lightgbm as lgb

import torch
import transformers as ppb

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Чтение данных

Прочитаем данные и сохраним в таблицу **df**.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

Выведем данные.

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет, нужно только обработать текст в признаках для моделей, которые не умеют этого делать сами: решающее дерево, случайный лес, логистическая регрессия...

## Обработка текста

Для начала лемматизируем текст и оставим в нем только слова без спец символов.

In [5]:
def lemmatize(text):
    m = WordNetLemmatizer()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text

In [6]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    text_arr = text.split()
    text = " ".join(text_arr)
    return text

In [7]:
df['lemm_text'] = df['text'].apply(lambda x: lemmatize(clear_text(x)))

## Обучение

Скачаем пакет стопслов.

In [8]:
f1_scorer = make_scorer(f1_score, average='binary')

In [9]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Bert

Мы будем использовать distilled версию bert, так как она быстрее и дешевле по памяти. Также эта модель уже предобучена на англоязычном корпусе.

Загрузка предобученной модели DistilBERT и токенизатора.

In [10]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Оставим только маленькие тексты, чтобы хватило памяти компьютера.

In [11]:
small_df = df[df['text'].apply(len) < 50]

In [12]:
small_df['toxic'].value_counts()

0    14300
1     3024
Name: toxic, dtype: int64

Закодируем строки, ограничив длину токена до 100. Кодирование происходит следующим образом: тексты разбиваются на токены, добавляются специальные токены и происходит замена каждого токена его идентификатором из таблицы эмбеддингов, которую мы получаем вместе с обученной моделью.

In [13]:
tokenized = small_df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 100, truncation = True))

Таким образом, мы получаем матрицу/тензор, который можно передавать BERT'у

Заполним недостающие до максимальной длины строки нулями в конце.

In [14]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)


padded = []
for i in tokenized.values:
    padded.append(i + [0]*(max_len - len(i)))

Порядок обработки входного вектора в DistilBERT'е точно такой же, как и в обычном BERT'е. На выходе будет вектор для каждого входного токена, состоящий из 768 чисел с плавающей точкой.

In [15]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids)

Поскольку перед нами стоит задача классификации предложений, мы игнорируем все, кроме первого вектора (связанного с [CLS] токеном). Этот вектор мы передаем в качестве входного в модель логистической регрессии.

In [16]:
features = last_hidden_states[0][:,0,:].numpy()

In [17]:
train_small_features, test_small_features, train_small_target, test_small_target = (
    train_test_split(features, small_df['toxic'], test_size=0.25))

Мы получили закодированные признаки, теперь нужно только обучить логистическую регрессию.

### Логистическая регрессия

In [18]:
%%time
#<Логистическая регрессия>
linear_parametrs = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

clf = LogisticRegression(max_iter = 10000, random_state = 12345)

grid = GridSearchCV(clf, linear_parametrs, cv=3, scoring = 'f1')
grid.fit(train_small_features, train_small_target)

print('linear regression:', abs(grid.best_score_))
print(grid.best_params_)

linear regression: 0.8075889010783186
{'C': 10}
Wall time: 3min 6s


Теперь обучим простые модели.

In [19]:
features_lemm = df['lemm_text']
features = df['text']

target = df['toxic']

train_features_lemm, test_features_lemm, train_target, test_target = (
    train_test_split(features_lemm, target, test_size=0.25))

Приведем все символы к типу Unicode и векторизуем их с помощью TfidfVectorizer.

In [20]:
train_features_vect = train_features_lemm.values.astype('U')
test_features_vect = test_features_lemm.values.astype('U')

#<min_df для исключения самых редких слов>
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df=0.0001)
train_features_vect = count_tf_idf.fit_transform(train_features_vect)
test_features_vect = count_tf_idf.transform(test_features_vect)

Обучим простые модели.

### Дерево решений

In [ ]:
%%time
#<дерево решений>
tree_parametrs = {'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

clf = DecisionTreeClassifier(random_state = 12345)

grid = GridSearchCV(clf, tree_parametrs, cv=3, scoring = 'f1')
grid.fit(train_features_vect, train_target)

print('tree:', abs(grid.best_score_))
print(grid.best_params_)

### Случайный лес

In [ ]:
%%time
#<случайный лес>
forest_parametrs = { 'n_estimators': range (10, 101, 10),
              'max_depth': range (1, 10)}

clf = RandomForestClassifier(random_state = 12345)

grid = GridSearchCV(clf, forest_parametrs, cv=3, scoring = 'f1')
grid.fit(train_features_vect, train_target)

print('forest:', abs(grid.best_score_))
print(grid.best_params_)

### Логистическая регрессия

In [ ]:
%%time
#<Логистическая регрессия>
linear_parametrs = {"C":np.logspace(-3,3,7)}

clf = LogisticRegression(max_iter = 1500, random_state = 12345)

grid = GridSearchCV(clf, linear_parametrs, cv=3, scoring = 'f1')
grid.fit(train_features_vect, train_target)

print('linear regression:', abs(grid.best_score_))
print(grid.best_params_)

### LightGBM

In [ ]:
%%time

#<LightGBM>
lgb_parametrs = {'num_leaves': range(10, 30, 10),
                 'max_depth': range(1, 10),
                 'n_estimators':range(10, 100, 10)}

clf = lgb.LGBMClassifier()

grid = GridSearchCV(clf, lgb_parametrs, cv=3, scoring = 'f1')
grid.fit(train_features_vect, train_target, verbose = 10)

print('LightGBM:', abs(grid.best_score_))
print(grid.best_params_)

## Тестирование

Обучим лучшие модели на на лучших параметрах.

In [25]:
df_results = pd.DataFrame(columns=['model', 'result'])

### Bert Логистическая регрессия

In [26]:
model = LogisticRegression(max_iter = 5000, random_state = 12345, C = 10.0)
model.fit(train_small_features, train_small_target)
predictions = model.predict(test_small_features)

df_results.loc[0] = (['Bert LogisticRegression', f1_score(test_small_target, predictions)])
f1_score(test_small_target, predictions)

0.8161865569272977



### Дерево решений

In [27]:
model = DecisionTreeClassifier(random_state = 12345, max_depth = 150)
model.fit(train_features_vect, train_target)
predictions = model.predict(test_features_vect)

df_results.loc[1] = (['DecisionTreeClassifier', f1_score(test_target, predictions)])
f1_score(test_target, predictions)

0.7226456178551985

### Логистическая регрессия

In [28]:
model = LogisticRegression(max_iter = 1000, random_state = 12345, C = 10.0)
model.fit(train_features_vect, train_target)
predictions = model.predict(test_features_vect)

df_results.loc[2] = (['LogisticRegression', f1_score(test_target, predictions)])
f1_score(test_target, predictions)

0.7759123074788401

### LightGBM

In [29]:
model = lgb.LGBMClassifier(max_depth = 9, n_estimators = 90, num_leaves = 20)
model.fit(train_features_vect, train_target)
predictions = model.predict(test_features_vect)

df_results.loc[3] = (['LGBMClassifier', f1_score(test_target, predictions)])
f1_score(test_target, predictions)

0.6635559131134353

### CatBoostClassifier

In [30]:
%%time


#<CatBoostClassifier>

model = CatBoostClassifier()
model.fit(train_features_vect.toarray(), train_target)

Learning rate set to 0.079478
0:	learn: 0.6127715	total: 1.48s	remaining: 24m 37s
1:	learn: 0.5459074	total: 2.16s	remaining: 18m
2:	learn: 0.4909971	total: 2.8s	remaining: 15m 29s
3:	learn: 0.4460436	total: 3.38s	remaining: 14m 1s
4:	learn: 0.4098122	total: 3.96s	remaining: 13m 7s
5:	learn: 0.3800263	total: 4.53s	remaining: 12m 30s
6:	learn: 0.3551523	total: 5.12s	remaining: 12m 5s
7:	learn: 0.3345333	total: 5.7s	remaining: 11m 47s
8:	learn: 0.3167946	total: 6.33s	remaining: 11m 36s
9:	learn: 0.3032729	total: 6.95s	remaining: 11m 28s
10:	learn: 0.2916704	total: 7.57s	remaining: 11m 20s
11:	learn: 0.2822595	total: 8.14s	remaining: 11m 10s
12:	learn: 0.2738068	total: 8.72s	remaining: 11m 2s
13:	learn: 0.2667461	total: 9.3s	remaining: 10m 55s
14:	learn: 0.2604170	total: 9.88s	remaining: 10m 48s
15:	learn: 0.2551964	total: 10.5s	remaining: 10m 43s
16:	learn: 0.2507552	total: 11s	remaining: 10m 38s
17:	learn: 0.2469628	total: 11.6s	remaining: 10m 33s
18:	learn: 0.2435121	total: 12.2s	remai

In [31]:
predictions = model.predict(test_features_vect.toarray())

df_results.loc[4] = (['CatBoostClassifier', f1_score(test_target, predictions)])
f1_score(test_target, predictions)

0.7551673944687045

## Выводы

In [32]:
df_results = df_results.set_index('model')
df_results.sort_values(by='result', ascending=False)

,result
model,
Bert LogisticRegression,0.816187
LogisticRegression,0.775912
CatBoostClassifier,0.755167
DecisionTreeClassifier,0.722646
LGBMClassifier,0.663556


Лучше всего справился Bert, хотя по моему мнению, если обучить его на более сложных текстах, качество еще вырастет. Но на данном этапе у нас нет таких ресурсов.